In [1]:
import os
import psycopg2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation


In [2]:
def conectar_db():
    return psycopg2.connect(
        host="localhost",
        port="5432",
        database="postgres",
        user="postgres",
        password="JONPER"
    )


In [3]:
def cargar_datos_gnss():
    query = """
    SELECT
        nombre_gnss,
        fecha_hora,
        azimut,
        desp_2d_mm,
        tasa_3d_mm_d
    FROM "MV_GNSS"."03_gnss_data_procesada"
    ORDER BY fecha_hora DESC;
    """

    conn = conectar_db()
    df = pd.read_sql(query, conn)
    conn.close()

    return df


In [4]:
def limpiar_datos(df, umbral_mm=0, fecha_inicio=None, fecha_fin=None):
    df = df.dropna(subset=["azimut", "desp_2d_mm"])
    df = df[df["desp_2d_mm"] > umbral_mm]
    
    # Filtrar por rango de fechas si se proporcionan
    if fecha_inicio:
        fecha_inicio = pd.to_datetime(fecha_inicio)
        df = df[df["fecha_hora"] >= fecha_inicio]
    
    if fecha_fin:
        fecha_fin = pd.to_datetime(fecha_fin)
        df = df[df["fecha_hora"] <= fecha_fin]
    
    print(f"Datos después de limpiar: {len(df)} registros")
    return df

In [5]:
def rose_diagram(ax, azimut_deg, weights=None, bins=16):
    theta = np.deg2rad(azimut_deg)

    hist, bin_edges = np.histogram(
        theta,
        bins=bins,
        weights=weights
    )

    width = bin_edges[1] - bin_edges[0]

    ax.bar(
        bin_edges[:-1],
        hist,
        width=width,
        align="edge",
        edgecolor="black",
        alpha=0.8
    )

    ax.set_theta_zero_location("N")
    ax.set_theta_direction(-1)

    ax.set_thetagrids(
        range(0, 360, 45),
        labels=["N","NE","E","SE","S","SW","W","NW"]
    )


In [6]:
def generar_rosetas_por_gnss(df, carpeta_salida="salida_rosetas"):
    os.makedirs(carpeta_salida, exist_ok=True)

    for gnss in df["nombre_gnss"].unique():
        dfg = df[df["nombre_gnss"] == gnss]
        
        if len(dfg) == 0:
            print(f"Sin datos para {gnss}")
            continue

        fig, ax = plt.subplots(
            subplot_kw={"projection": "polar"},
            figsize=(6,6)
        )

        rose_diagram(
            ax,
            dfg["azimut"],
            weights=dfg["desp_2d_mm"]
        )

        ax.set_title(f"Roseta direccional {gnss}")

        plt.savefig(
            f"{carpeta_salida}/roseta_{gnss}.png",
            dpi=300,
            bbox_inches="tight"
        )
        plt.close()
        print(f"Roseta guardada: {gnss}")

In [7]:
def roseta_evento(df, fecha_evento, gnss):
    df_g = df[df["nombre_gnss"] == gnss]
    
    if isinstance(fecha_evento, str):
        fecha_evento = pd.to_datetime(fecha_evento)

    df_before = df_g[df_g["fecha_hora"] < fecha_evento]
    df_after  = df_g[df_g["fecha_hora"] >= fecha_evento]
    
    if len(df_before) == 0 or len(df_after) == 0:
        print(f"Datos insuficientes: antes={len(df_before)}, después={len(df_after)}")
        return

    fig, axes = plt.subplots(
        1, 2,
        subplot_kw={"projection": "polar"},
        figsize=(10,5)
    )

    rose_diagram(
        axes[0],
        df_before["azimut"],
        weights=df_before["desp_2d_mm"]
    )
    axes[0].set_title("Antes del evento")

    rose_diagram(
        axes[1],
        df_after["azimut"],
        weights=df_after["desp_2d_mm"]
    )
    axes[1].set_title("Después del evento")

    plt.show()

In [8]:
def animar_roseta(df, gnss, carpeta_salida="salida_rosetas"):
    os.makedirs(carpeta_salida, exist_ok=True)
    
    df_g = df[df["nombre_gnss"] == gnss]
    
    if len(df_g) == 0:
        print(f"Sin datos para {gnss}")
        return

    fig, ax = plt.subplots(
        subplot_kw={"projection": "polar"},
        figsize=(6,6)
    )

    fechas = sorted(df_g["fecha_hora"].unique())

    def update(frame):
        ax.clear()
        dft = df_g[df_g["fecha_hora"] <= frame]
        if len(dft) > 0:
            rose_diagram(
                ax,
                dft["azimut"],
                weights=dft["desp_2d_mm"]
            )
        ax.set_title(f"{gnss} hasta {pd.to_datetime(frame).date()}")

    ani = FuncAnimation(
        fig,
        update,
        frames=fechas,
        interval=200,
        repeat=True
    )

    # Guardar como gif (más compatible que mp4)
    output_path = f"{carpeta_salida}/animacion_{gnss}.gif"
    ani.save(output_path, writer='pillow', dpi=100)
    print(f"Animación guardada: {output_path}")
    plt.close()

In [9]:
def main():
    df = cargar_datos_gnss()
    
    # OPCIÓN 1: Todos los datos
    # df = limpiar_datos(df)
    
    # OPCIÓN 2: Solo un año completo
    # df = limpiar_datos(df, fecha_inicio="2024-01-01", fecha_fin="2024-12-31")
    
    # OPCIÓN 3: Un rango específico de meses
    df = limpiar_datos(df, fecha_inicio="2025-10-20", fecha_fin="2025-12-11")
    
    # OPCIÓN 4: Con umbral de desplazamiento
    # df = limpiar_datos(df, umbral_mm=0.5, fecha_inicio="2024-01-01", fecha_fin="2024-12-31")

    generar_rosetas_por_gnss(df)

    # Ejemplo evento 
    roseta_evento(df, "2025-12-25", "GNSS01")

    # Ejemplo animación - ACTIVADO (sin guión en el nombre)
    # animar_roseta(df, "GNSS01")

if __name__ == "__main__":
    main()

C:\Users\jonat\AppData\Local\Temp\ipykernel_23076\2563646529.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Datos después de limpiar: 4509 registros
Roseta guardada: GNSS04
Roseta guardada: GNSS01
Roseta guardada: GNSS03
Roseta guardada: GNSS02
Datos insuficientes: antes=1066, después=0
Animación guardada: salida_rosetas/animacion_GNSS01.gif
